In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
food = ["라면", "누드김밥", "빨계떡", "계란볶음밥"]
food_ex = ["한국인이 가장 많이 먹고 즐겨찾는 주식! 라면입니다!",
"건강한 재료로 가득담아 말은 누드김밥! 고기가 없어도 맛있어요!",
"매콤한 고춧가루와 시원한 파가 섞인 빨계떡 라면입니다. 해장에도 도움이 됩니다.",
"토마토와 달걀을 넣은 볶음밥에 카레소스를 두른 영양만점 볶음밥"]
df = pd.DataFrame(list(zip(food, food_ex)),columns=['food','food_ex'])
df

,food,food_ex
0,라면,한국인이 가장 많이 먹고 즐겨찾는 주식! 라면입니다!
1,누드김밥,건강한 재료로 가득담아 말은 누드김밥! 고기가 없어도 맛있어요!
2,빨계떡,매콤한 고춧가루와 시원한 파가 섞인 빨계떡 라면입니다. 해장에도 도움이 됩니다.
3,계란볶음밥,토마토와 달걀을 넣은 볶음밥에 카레소스를 두른 영양만점 볶음밥


In [4]:
tfidf_vector = TfidfVectorizer()
#tfidf_vector = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix = tfidf_vector.fit_transform(df['food_ex']).toarray()
#tfidf_matrix = tfidf_vector.fit_transform(movie_data['genres']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names()

In [5]:
tfidf_matrix.shape

(4, 32)

In [6]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = df.food)
print(tfidf_matrix.shape)
tfidf_matrix.head()

(4, 32)


,가득담아,가장,건강한,고기가,고춧가루와,넣은,누드김밥,달걀을,도움이,됩니다,...,없어도,영양만점,재료로,주식,즐겨찾는,카레소스를,토마토와,파가,한국인이,해장에도
food,,,,,,,,,,,,,,,,,,,,,
라면,0.000000,0.388614,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.388614,0.388614,0.000000,0.000000,0.000000,0.388614,0.000000
누드김밥,0.353553,0.000000,0.353553,0.353553,0.000000,0.000000,0.353553,0.000000,0.000000,0.000000,...,0.353553,0.000000,0.353553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
빨계떡,0.000000,0.000000,0.000000,0.000000,0.322386,0.000000,0.000000,0.000000,0.322386,0.322386,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.322386,0.000000,0.322386
계란볶음밥,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,0.353553,0.000000,0.000000,...,0.000000,0.353553,0.000000,0.000000,0.000000,0.353553,0.353553,0.000000,0.000000,0.000000


In [7]:
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim.shape

(4, 4)

In [8]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = df.food, columns = df.food)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(4, 4)


food,라면,누드김밥,빨계떡,계란볶음밥
food,,,,
라면,1.000000,0.0,0.077876,0.0
누드김밥,0.000000,1.0,0.000000,0.0
빨계떡,0.077876,0.0,1.000000,0.0
계란볶음밥,0.000000,0.0,0.000000,1.0


In [10]:
def genre_recommendations(target_title, matrix, items, k=1):
    recom_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_title = items.iloc[recom_idx, :].food.values
    recom_genre = items.iloc[recom_idx, :].food_ex.values
    target_title_list = np.full(len(range(k)), target_title)
    target_genre_list = np.full(len(range(k)), items[items.food == target_title].food_ex.values)
    d = {
        'target_title':target_title_list,
        'target_genre':target_genre_list,
        'recom_title' : recom_title,
        'recom_genre' : recom_genre
    }
    return pd.DataFrame(d)
genre_recommendations('라면', cosine_sim_df, df)

,target_title,target_genre,recom_title,recom_genre
0,라면,한국인이 가장 많이 먹고 즐겨찾는 주식! 라면입니다!,빨계떡,매콤한 고춧가루와 시원한 파가 섞인 빨계떡 라면입니다. 해장에도 도움이 됩니다.
